In [2]:
#coding=utf-8
import sys
import os
import copy


sys.path.insert(0, "../")
from common import get_tb_info
from common import debug_line

%load_ext Cython

# <font color=blue size=72>一、第一部分</font>

## <font color=blue>1.1 cython的速度提升</font> 

In [3]:
%%cython
import numpy as np
cimport numpy as cnp

from scipy.stats import poisson
poi = poisson(10.0)
n = 100
pmf = poi.pmf(np.arange(n))
print("pmf.type",type(pmf))

def shannon_entropy_cy(cnp.ndarray p_x):
    return - np.sum(p_x * np.log(p_x))

def shannon_entropy_py(p_x):
    return - np.sum(p_x * np.log(p_x))

pmf.type <class 'numpy.ndarray'>


In [ ]:
%%timeit
poi.entropy()

In [ ]:
%%timeit
shannon_entropy_py(pmf)

In [ ]:
%%timeit
shannon_entropy_cy(pmf)

In [ ]:
print(poi.entropy(),"|", shannon_entropy_py(pmf), "|",shannon_entropy_cy(pmf))

In [ ]:
%%cython

cimport numpy as cnp
from libc.math cimport log as clog

cpdef shannon_entropy_v1(cnp.ndarray p_x):
    cdef double res = 0.0
    cdef int n = p_x.shape[0]
    cdef int i
    for i in range(n):
        res += p_x[i] * clog(p_x[i])
    return -res

In [ ]:
%%timeit
shannon_entropy_v1(pmf)

In [ ]:
%%cython

cimport numpy as cnp
from libc.math cimport log as clog

cpdef shannon_entropy_v2(cnp.ndarray[double] p_x):
    cdef double res = 0.0
    cdef int n = p_x.shape[0]
    cdef int i
    for i in range(n):
        res += p_x[i] * clog(p_x[i])
    return -res

In [ ]:
%%timeit
shannon_entropy_v2(pmf)

In [ ]:
%%cython

cimport cython
cimport numpy as cnp
from libc.math cimport log

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef shannon_entropy_v3(cnp.ndarray[double] p_x):
    cdef double res = 0.0
    cdef int n = p_x.shape[0]
    cdef int i
    for i in range(n):
        res += p_x[i] * log(p_x[i])
    return -res

In [ ]:
%%timeit
shannon_entropy_v3(pmf)

In [ ]:
%%cython 

cimport cython
from libc.math cimport log

@cython.boundscheck(False)
@cython.wraparound(False)
def shannon_entropy_v4(double[::1] p_x):
    cdef double res = 0.0
    cdef int n = p_x.shape[0]
    cdef int i
    for i in range(n):
        res += p_x[i] * log(p_x[i])
    return -res

In [ ]:
%%timeit
shannon_entropy_v4(pmf)

In [ ]:
print(poi.entropy())
print(shannon_entropy_v1(pmf))
print(shannon_entropy_v2(pmf))
print(shannon_entropy_v3(pmf))
print(shannon_entropy_v4(pmf))

## <font color=blue>1.2 profiler，更详细的代码见笔记本linshi_env</font> 

In [ ]:
from line_profiler import LineProfiler
import random

def do_stuff(numbers):
    s = sum(numbers)
    l = [numbers[i]/43 for i in range(len(numbers))]
    m = ['hello'+str(numbers[i]) for i in range(len(numbers))]
    return m    



numbers = [i for i in range(10000)]
lp = LineProfiler()
lp_wrapper = lp(do_stuff)
lp_wrapper(numbers)
lp.print_stats()

In [ ]:
%%timeit
do_stuff(numbers)

In [ ]:
import cProfile
import re
cProfile.run('re.compile("foo|bar")')

## <font color=blue>1.3 cython与c的连接，见connect_c文件夹</font> 

## <font color=blue>1.4 并行-ray的语法，见ray文件夹</font> 

## <font color=blue>1.5 openmp的语法【unfix】</font> 

## <font color=blue>1.6 eigen的语法【unfix】</font> 

## <font color=blue>1.7 onetbb的语法【unfix】</font> 

## <font color=blue>1.8 dataframe <=> cnumpy.ndarray</font> 

In [ ]:
%%cython

import numpy as np
cimport numpy as cnp
import pandas as pd


data_dict = {}
data_dict["x"] = [1,2,3]
data_dict["y"] = [11,12,13]

data_df = pd.DataFrame(data=data_dict)
print(data_df)
print(data_df.values)

cdef int[:,:] aaa = data_df.values

In [5]:
aaa = [1,2,3]
bbb = data_df.values

In [7]:
bbb[0]

array([ 1, 11])

In [7]:
%%cython
cimport numpy as cnp
cdef cnp.ndarray ccc;